# สำรวจชุดข้อมูลดอกไอริส

**จุดประสงค์การเรียนรู้**

1. ทราบความแตกต่างระหว่าง Mean กับ Median และ SD กับ IQR
2. สามารถวาด Box plot (และ Violin plot) ได้ และเข้าใจความหมายของ Box plot (และ Violin plot)
3. สามารถวาด Histogram ได้ และเข้าใจความหมายของ Histogram
4. เข้าใจการคำนวณ Correlation และการตีความค่า Correlation

## ค่าสถิติเบื้องต้น

### ค่าสถิติที่แสดงค่ากลาง: Mean vs. Median

ก่อนที่จะเข้าสู่บทเรียนวันนี้ ขอทบทวนความรู้ทางสถิติว่า Mean กับ Median ต่างกันอย่างไร

สมมติว่ามีข้อมูลเงินเดือนของคน 10 คน เรียงลำดับจากน้อยไปมาก ดังนี้

|คนที่ |1|2|3|4|5|6|7|8|9|10|
|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|
|เงินเดือน (หมื่นบาท)|0.3| 0.5| 0.6| 0.7| 1.0| 1.2| 4.0| 9.0| 10.0| 100.0 |

โดยข้อมูลเก็บในตัวแปรชื่อ `salary`

In [ ]:
import pandas as pd
salary = pd.Series([0.3, 0.5, 0.6, 0.7, 1.0, 1.2, 4.0, 4.0, 5.0, 100.0])

จะเห็นว่าคำนวณ mean และ median ได้ดังนี้

In [ ]:
print(salary.agg(['mean','median']))

mean      11.73
median     1.10
dtype: float64


ซึ่งพบว่า median เป็นตัวแทนข้อมูลได้ดีกว่า mean เนื่องจากในข้อมูลมีคนที่มีเงินเดือน 100 ที่เป็นข้อมูลสุดโต่ง (outlier) อยู่

ดังนั้น ทางแก้คือ
- เลือกใช้ median แทน mean
- หากใช้ mean ก็ต้องลบ outlier ออกก่อนการคำนวณ mean

### ค่าสถิติแสดงการกระจายตัว: SD vs. IQR

จากข้อมูล `salary` ข้างต้น พบว่า เมื่อข้อมูลมี outlier ค่า SD ก็ไม่สามารถเป็นตัวแทนข้อมูลที่ดีเช่นกัน  

In [ ]:
print(salary.agg(['std']))

std    31.063914
dtype: float64


ดังนั้น จึงเกิดค่าสถิติที่เรียกว่า Interquartile Range (IQR) ซึ่งมีที่มาในลักษณะเดียวกันกับ median แต่ใช้ในการวัดการกระจายตัวของข้อมูล

สำหรับค่า IQR คำนวณได้จาก

$$
IQR = Q_3 - Q_1 = P_{75} - P_{25}
$$

โดยที่
- $Q_1$ และ $Q_3$ คือควอร์ไทล์ที่ 1 และ 3 ตามลำดับ
- $P_{25}$ และ $P_{75}$ คือเปอร์เซนต์ไทล์ที่ 25 และ 75 ตามลำดับ

(*หมายเหตุ: วิชานี้ จะไม่ลงรายละเอียดการคำนวณควอล์ไทล์หรือเปอร์เซนต์ไทล์*)

ซึ่งจากสูตร $IQR$ จะสามารถคำนวณค่า $IQR$ จากข้อมูล ได้ดังนี้

In [ ]:
p25 = salary.quantile(0.25)
p75 = salary.quantile(0.75)
iqr = p75 - p25
print(f'IQR = {iqr}')

IQR = 3.375


จะเห็นว่า IQR เป็นตัวแทนข้อมูลได้ดีกว่า SD ในกรณีที่มี outlier

## ชุดข้อมูลดอกไอริส

ประวัติศาสตร์ของชุดข้อมูลดอกไอริสนี้ ดูได้จาก Wikipedia: https://en.wikipedia.org/wiki/Iris_flower_data_set

<table>
<tr>
<td>Setosa</td>
<td>Virginica</td>
<td>Versicolor</td>
</tr>
<tr><td>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a7/Irissetosa1.jpg/640px-Irissetosa1.jpg" width=200px>
</td>
<td>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Iris_virginica.jpg/590px-Iris_virginica.jpg" width=200px>
</td>
<td>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Iris_versicolor_3.jpg/640px-Iris_versicolor_3.jpg" width=200px>
</td>
</tr>
</table>


ก่อนอื่นดาวน์โหลดชุดข้อมูลดอกไอริสจาก UCI repository และตั้งชื่อเป็น `iris.zip`

In [ ]:
!wget -c "https://archive.ics.uci.edu/static/public/53/iris.zip" -O iris.zip

--2023-12-18 16:04:26--  https://archive.ics.uci.edu/static/public/53/iris.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘iris.zip’

iris.zip                [ <=>                ]   3.65K  --.-KB/s    in 0s      

2023-12-18 16:04:26 (717 MB/s) - ‘iris.zip’ saved [3738]



จากนั้นทำการ unzip ไฟล์ด้วยคำสั่ง `unzip` โดยกำหนด option `-d <ชื่อโฟลเดอร์>` เพื่อระบุโฟลเดอร์ปลายทางที่ต้องการเก็บไฟล์ ซึ่งในที่นี้ จะกำหนดให้บันทึกไว้ในโฟลเดอร์ชื่อ `iris_data` และ option `-o` หมายถึง ให้ unzip เขียนทับไฟล์ที่มีอยู่แล้ว

In [ ]:
!unzip -o iris.zip -d iris_data

Archive:  iris.zip
  inflating: iris_data/Index         
  inflating: iris_data/bezdekIris.data  
  inflating: iris_data/iris.data     
  inflating: iris_data/iris.names    


ข้อมูลของดอกไอริสจะถูกเก็บไว้ในไฟล์ `iris_data/iris.data` โดยไฟล์ `iris_data/iris.names` จะเป็นที่เป็นคำอธิบายชุดข้อมูล

Data dictionary ของชุดข้อมูลในไฟล์ `iris_data/iris.data` เป็นดังนี้

| Attribute | Description |
|:--|:--|  
|1| ความยาวกลีบเลี้ยง (sepal length) หน่วย cm |
|2| ความกว้างกลีบเลี้ยง (sepal width) หน่วย cm |
|3| ความยาวกลีบดอก (petal length) หน่วย cm |
|4| ความกว้า่งกลีบดอก (petal width) หน่วย cm |
|5| สายพันธุ์ หรือ class ของดอก ซึ่งเป็นไปได้ 3 แบบคือ Iris Setosa, Iris Versicolour, หรือ Virginica |

ข้อมูลใน `iris_data/iris.data` มีลักษณะเป็นไฟล์ csv เพียงแค่ไม่ได้ใช้นามสกุล `.csv` ดังนั้น จึงจะใช้ `pd.read_csv()` ในการอ่านข้อมูลเข้ามา

In [ ]:
import pandas as pd
iris = pd.read_csv('iris_data/iris.data')  # ผิด เพราะอ่านไม่ครบทุกแถว
iris

,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa
...,...,...,...,...,...
144,6.7,3.0,5.2,2.3,Iris-virginica
145,6.3,2.5,5.0,1.9,Iris-virginica
146,6.5,3.0,5.2,2.0,Iris-virginica
147,6.2,3.4,5.4,2.3,Iris-virginica


โค้ดข้างต้นอ่านข้อมูลมาได้ 149 แถว ซึ่งไม่ถูกต้อง เนื่องจากข้อมูลดอกไอริสจริง ๆ มี 150 แถว ซึ่งสาเหตุของความผิดพลาดมาจากการที่ไฟล์ `iris_data/iris.data` ไม่มีแถวแรกที่เก็บชื่อคอลัมน์ (header) ดังนั้น จึงแก้ไขด้วยการกำหนดพารามิเตอร์ `header=None` แล้วตั้งชื่อคอลัมน์เองตาม Data dictionary

In [ ]:
iris = pd.read_csv('iris_data/iris.data', header=None)
iris

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [ ]:
iris.columns = ['sepal_length',
                'sepal_width',
                'petal_length',
                'petal_width',
                'class']
iris

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


**หมายเหตุ:** อาจใช้คำสั่ง `DataFrame.rename()` ในการเปลี่ยนชื่อคอลัมน์ก็ได้ ดังนี้ ซึ่งจะได้ผลลัพธ์เหมือนกัน

```
mapper = {0:'sepal_length',
          1:'sepal_width',
          2:'petal_length',
          3:'petal_width',
          4:'class'}
iris = iris.rename(mapper, axis=1)   # อาจใช้ inplace=True ก็ได้หากต้องการเขียนทับ
```


จากข้อมูลดอกไอริส สามารถดูค่าสถิติง่าย ๆ ได้ ดังนี้

In [ ]:
iris.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


## Box plot (กราฟรูปกล่อง)

แม้ว่าคำสั่ง `iris.describe()` สามารถสรุปค่าสถิติออกมาคือ mean, std, min, 25%, 50%, 75% และ max แต่มันก็ยากที่จะเห็นภาพ  ดังนั้นจึงเกิดกราฟชนิดหนึ่งที่เรียกว่า Box plot ที่ใช้ในการสรุปค่าสถิติเหล่านี้ในกราฟเดียว



### Box plot แบบพื้นฐาน

ในขั้นต้น จะยกตัวอย่างการวาดกราฟ Box plot ของความยาวกลีบเลี้ยง (sepal length) โดยใช้ plotly ดังนี้


In [ ]:
import plotly.express as px
px.box(iris, x=['sepal_length'], orientation='h')

จากกราฟหากเอา mouse ไปวางอยู่ตำแหน่งบนกราฟ จะเห็นว่าค่าองค์ประกอบของ Box plot ของความยาวกลีบเลี้ยง (sepal length)  ที่ได้ (เรียงลำดับจากซ้ายไปขวา) ถูกสร้างมาจากค่า

| ค่า | ความหมาย | ค่า |
|:--|:--|:--|
| **min** | ค่าต่ำสุด ($Q_0$ หรือ 0th percentile) |  4.3 |
| **q1** (25%) | ค่า First quartile ($Q_1$ หรือ 25th percentile) | 5.1 |
| **median** (50%) | ค่ามัธยฐาน ($Q_2$ หรือ 50th percentile) | 5.8 |
| **q3** (75%) | ค่า Third quartile ($Q_3$ หรือ 75th percentile) | 6.4 |
| **max** | ค่าสูงสุด ($Q_4$ หรือ 100th percentile)| 7.9 |

ซึ่งตรงกับคำสั่ง `iris.describe()` ของ Pandas

Box plot มีชื่อเรียกอีกอย่างว่า **Box-and-whisker plot** เนื่องจากลักษณะของภาพที่ประกอบด้วย 2 ส่วนคือ กล่อง (Box) และหนวด (Whisker)

- **Box** สร้างจากค่า $Q_1$ และ $Q_3$ โดยมีเส้นขีดขวางแสดงค่า $Q_2$
- **Whisker** แสดงถึงขอบเขตค่าของข้อมูล ซึ่งในกรณีของรูป Box plot ข้างต้น จะแสดง Whisker อยู่ 2 เส้นคือ
  - เส้นที่ลากจากค่า $Q_1$ ถึง min
  - เส้นที่ลากจากค่า $Q_3$ ถึง max



### Box plot แบบมี outlier

ทีนี้ ลองเปลี่ยนไปวาดกราฟของความกว้างกลีบเลี้ยง (sepal width) ดูบ้าง พบว่าได้ Box pot ดังนี้

In [ ]:
px.box(iris, x=['sepal_width'], orientation='h')

จากกราฟจะเห็นว่า แม้ว่าค่าสถิติ (min/q1/median/q3/max) ได้เหมือนกับค่าจากคำสั่ง `iris.describe()` แต่องค์ประกอบของ Box plot ของ sepal width มีความแตกต่างจากของ sepal length อยู่บ้าง เนื่องจาก Whisker ทั้งสองเส้น ลากไปไม่ถึงค่า min กับค่า max

กล่าวคือ
- ค่า min กับค่า low fence เป็นคนละค่ากัน และ
- ค่า max กับค่า upper fence ก็เป็นคนละค่ากัน ดังนี้

| ค่า | ความหมาย | ค่า |
|:--|:--|:--|
| **min** | ค่าต่ำสุด ($Q_0$ หรือ 0th percentile) |  2 |
| **lower fence**  | ? | **2.2**|
| **q1** (25%) | ค่า First quartile ($Q_1$ หรือ 25th percentile) | 2.8 |
| **median** (50%) | ค่ามัธยฐาน ($Q_2$ หรือ 50th percentile)| 3 |
| **q3** (75%) | ค่า Third quartile ($Q_3$ หรือ 75th percentile) | 3.3 |
| **upper fence** | ? |**4** |
| **max** | ค่าสูงสุด ($Q_4$ หรือ 100th percentile)| 4.4 |

นอกจากนี้ ใน Box plot จะเห็นว่ามีจุดข้อมูลเดี่ยว ๆ แสดงอยู่นอกช่วงของ Box กับ Whisker อยู่ด้วย เราเรียกจุดข้อมูลเหล่านี้ว่า **Outlier**

**ค่า lower fence กับค่า upper fence คือค่าอะไร?**

กำหนดให้ $IQR$ คือ ค่า Interquartile range ซึ่งคำนวณได้จาก

$$
IQR = Q_3 - Q_1
$$

ซึ่งก็คือความยาวของกล่องของ Box plot นั่นเอง

จากค่า IQR สามารถคำนวณ lower fence และ upper fence ได้ ดังนี้

1. lower fence คือจุดข้อมูลที่น้อยที่สุด ที่มีค่ามากกว่าหรือเท่ากับ $Q_1 - 1.5 \times IQR$
2. upper fence คือจุดข้อมูลที่มากที่สุด ที่มาค่าน้อยกว่าหรือเท่ากับ $Q_3 + 1.5 \times IQR$

เพื่อทำความเข้าใจ lower/upper fences เราจะลองคำนวณค่าเอง เปรียบเทียบกับค่าที่แสดงใน Plotly ดังนี้

In [ ]:
q1 = iris['sepal_width'].quantile(0.25)  # ฟังก์ชันคำนวณ 25th percentile
q3 = iris['sepal_width'].quantile(0.75)  # ฟังก์ชันคำนวณ 75th percentile
iqr = q3 - q1
print(f'q1 = {q1}, q3 = {q3}, iqr = q3 - q1 = {iqr}')
print(f'คำนวณ q1-1.5*iqr ได้เท่ากับ {q1 - 1.5*iqr}')
print(f'คำนวณ q3+1.5*iqr ได้เท่ากับ {q3 + 1.5*iqr}')

q1 = 2.8, q3 = 3.3, iqr = q3 - q1 = 0.5
คำนวณ q1-1.5*iqr ได้เท่ากับ 2.05
คำนวณ q3+1.5*iqr ได้เท่ากับ 4.05


จากข้อมูล sepal width คำนวณได้ว่า
- $Q_1 - 1.5 \times IQR = 2.05$
- $Q_3 + 1.5 \times IQR = 4.05$

ค่านี้ ไม่ใช่ค่า lower fence และ upper fence ในทันที แต่ต้องใช้ค่าจากข้อมูลจริง ๆ ซึ่งในกรณีนี้ Plotly พบว่า
- จุดข้อมูลที่น้อยที่สุดที่มากกว่า 2.05 คือ sepal width=2.2 จึงได้ lower fence = 2.2
- จุดข้อมูลที่มากที่สุดที่น้อยกว่า 4.05 คือ sepal width=4.0 จึงได้ upper fence = 4.0

แต่ข้อมูลเป็นอย่างนั้นจริง ๆ หรือ? เราสามารถแสดงค่าข้อมูลจริง ควบคู่ไปกับ Box plot ได้โดยให้กำหนด `points='all'` ดังนี้ ซึ่งจากกราฟด้านล่างนี้ จะเห็นจุดข้อมูลจริงที่มีค่าเท่ากับ 2.2 และ 4.5 อยู่ที่ตำแหน่ง lower fence และ upper fence ตามลำดับ

In [ ]:
px.box(iris, x=['sepal_width'], orientation='h', points='all')

### ทำไมจึงมองว่าข้อมูลนอกช่วง Lower/Upper fence เป็น outlier

ภาพจาก Wikipedia แสดงให้เห็นว่า หากข้อมูลมีการแจกแจงแบบระฆังคว่ำ (Normal distribution) ที่มี

- ค่าเฉลี่ย ($\mu$) เท่ากับ $0$ และ
- ส่วนเบี่ยงเบนมาตรฐาน ($\sigma$) เท่ากับ 1 แล้ว

จะสามารถวาด Box plot ได้ในลักษณะดังรูป

<img src="https://upload.wikimedia.org/wikipedia/commons/1/1a/Boxplot_vs_PDF.svg">

จากคุณสมบัติของกราฟระฆังคว่ำสมมาตร จะมีค่ามัธยฐานเท่ากับค่าเฉลี่ย ดังนั้น Box plot ที่ได้ จึงสมมาตรซ้ายขวาที่ตำแหน่ง $0\sigma$

จากกราฟ เมื่อพิจารณาพื้นที่ใต้กราฟ จะตีความได้ว่า
- ความน่าจะเป็นที่ข้อมูลจะอยู่ในช่วง $Q_1$ ถึง $Q_3$ คือ $50\%$
- ความน่าจะเป็นที่ข้อมูลจะอยู่ในช่วง $Q_1 - 1.5\times IQR$ ถึง $Q_1$ คือ $24.65\%$
- ความน่าจะเป็นที่ข้อมูลจะอยู่ในช่วง $Q_3$ ถึง $Q_3 + 1.5\times IQR$ คือ $24.65\%$


สำหรับค่าสุดโต่งที่อยู่นอกช่วงของ Box และ Whisker คือ

1. ค่าข้อมูลที่น้อยกว่า $Q_1 - 1.5\times IQR$ คือข้อมูลที่มีค่าน้อยกว่า $-2.689\sigma$
2. ค่าข้อมูลที่มากกว่า $Q_3 + 1.5\times IQR$ คือข้อมูลที่มีค่ามากกว่า $+2.689\sigma$

ซึ่งแต่ละกรณีนี้ มีความน่าจะเป็นของการเกิดข้อมูลเหล่านี้คือ

$$
\frac{100\% - (24.65\% + 50\% + 24.65\%)}{2} = 0.35\%
$$

หรือก็คือ 0.35 ใน 100 ซึ่งถือว่าน้อยมาก เราจึงถือว่ามันเป็นข้อมูลที่(อาจจะ)เป็น outlier

*หมายเหตุ:* โดยปกติ เรามักจะถือว่าค่าที่อยู่นอกช่วง $\mu\pm 2\sigma$ เป็น outlier ซึ่งสามารถตัดออกจากการวิเคราะห์ข้อมูลได้ แต่สำหรับ Box plot จะเทียบได้กับค่า $\mu\pm 2.689\sigma$

### แสดง Mean และ SD ใน Box plot

**แสดงค่าเฉลี่ย**

In [ ]:
fig = px.box(iris, y=['sepal_width'], orientation='v')
fig.update_traces(boxmean=True)

**แสดงค่าเฉลี่ย และส่วนเบี่ยงเบนมาตรฐาน**

In [ ]:
fig = px.box(iris, y=['sepal_width'], orientation='v')
fig.update_traces(boxmean='sd')

### Box plot แบบต่าง ๆ

#### ตัวอย่างการวาดหลาย Box plot พร้อมกัน

In [ ]:
px.box(iris,
       x=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
       orientation='h')

#### ตัวอย่างการวาด Box plot แนวตั้ง

In [ ]:
px.box(iris,
       y=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
       orientation='v')

#### ตัวอย่างการวาดจุดข้อมูลแยกออกมา

In [ ]:
px.box(iris,
       y=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
       orientation='v',
       points='all')

#### ตัวอย่างการวาด Box plot แต่ไม่แสดง Box plot

ในกรณีที่ต้องการแต่จุดข้อมูล แต่ไม่ต้องการแสดง Box และ Whisker ก็สามารถทำได้ แต่ต้องใช้คำสั่ง `px.strip()` แทน

In [ ]:
px.strip(iris,
       y=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
       orientation='v')

#### ตัวอย่างการวาด Box plot เป็นกลุ่มตามสายพันธุ์

Box plot ที่ผ่านมา เป็นการวาด Box plot ของทุกสายพันธ์รวมกัน เพียงแต่แยกตาม Attribute ของดอก

หากต้องการวาด Box plot ให้แยกตามสายพันธุ์ของดอกด้วย ก็สามารถเพิ่มมิติของสีเข้าไปด้วย

In [ ]:
px.box(iris,
       y=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
       orientation='v',
       points='all',
       color='class')

In [ ]:
px.strip(iris,
       y=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
       color='class',
       orientation='v')

#### ตัวอย่างการเปรียบเทียบค่า `sepal_length` ระหว่างสายพันธ์ต่าง ๆ



In [ ]:
px.box(iris,
       y='sepal_length',
       color='class')

กราฟข้างต้น แกนนอน (x) เป็นแกนของสายพันธุ์ (class) ซึ่งมีความซ้ำซ้อนกับคำอธิบายสีของกราฟ (legend) ทางด้านขวามือ

เราสามารถเปลี่ยนค่าในแกนนอนเป็นค่าอื่น ๆ ได้ตามต้องการ ยกตัวอย่างเช่น หากพิจารณาค่า `petal_length` ตามเงื่อนไขดังนี้

| ช่วงของค่า | ชื่อกลุ่ม |
|:--|:--|
| `petal_length < 5` | `Small` |
| `petal_length >= 5`  | `Large` |

ดอกไอริสแต่ละดอก ก็จะถูกจัดเป็นกลุ่ม `Small` หรือ `Large` โดยค่าสมาชิกกลุ่มเหล่านี้ กำหนดให้เก็บในคอลัมน์ที่สร้างใหม่ ชื่อ `petal_length_group` ดังโค้ดต่อไปนี้

In [ ]:
# สร้างคอลัมน์ใหม่
iris['petal_length_group'] = (iris['petal_length'] >= 5)
iris['petal_length_group'] = iris['petal_length_group'].replace(
    {True: 'Large', False: 'Small'}
    )
iris

,sepal_length,sepal_width,petal_length,petal_width,class,petal_length_group
0,5.1,3.5,1.4,0.2,Iris-setosa,Small
1,4.9,3.0,1.4,0.2,Iris-setosa,Small
2,4.7,3.2,1.3,0.2,Iris-setosa,Small
3,4.6,3.1,1.5,0.2,Iris-setosa,Small
4,5.0,3.6,1.4,0.2,Iris-setosa,Small
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica,Large
146,6.3,2.5,5.0,1.9,Iris-virginica,Large
147,6.5,3.0,5.2,2.0,Iris-virginica,Large
148,6.2,3.4,5.4,2.3,Iris-virginica,Large


จากข้อมูลข้างต้น สามารถนำมาสร้าง Box plot ที่แกนนอนจำแนกตามกลุ่ม `Small` หรือ `Large` ได้ดังนี้

In [ ]:
px.box(iris,
       y='sepal_length',
       x='petal_length_group',
       color='class')

**ลองทำ:** ในโค้ดข้างต้น ลองเปลี่ยนจุดแบ่งขนาดของ `petal_length` จาก `5` เป็น `2.5` ดังนี้ แล้วดูว่ากราฟที่ได้ เป็นอย่างไร จงพิจารณาว่า เหตุใดจึงเป็นเช่นนั้น

| ช่วงของค่า | ชื่อกลุ่ม |
|:--|:--|
| `petal_length < 2.5` | `Small` |
| `petal_length >= 2.5`  | `Large` |

**ข้อเสียของ Box plot**
- แทนข้อมูล attribute ใด ๆ ด้วยค่าเพียงไม่กี่ค่า (min/$Q_1$/$Q_2$/$Q_3$/max) ดังนั้นจึงมีความเป็นไปได้ที่จะมี Box plot เหมือนกัน แต่มาจากข้อมูลที่มีการแจกแจงต่างกัน

**แหล่งข้อมูล Box plot ที่เกี่ยวข้อง**
- Wikipedia: https://en.wikipedia.org/wiki/Box_plot
- ตัวอย่าง Box plot จากเว็บของ Plotly https://plotly.com/python/box-plots/
- คู่มืออ้างอิงคำสั่ง `px.box()` https://plotly.com/python-api-reference/generated/plotly.express.box

## Violin plot

เป็นกราฟที่มีรูปแบบคล้าย Box plot แต่จะมีการใช้เทคนิค Kernel density estimation มาใช้ประมาณ Histogram ให้เป็นเส้นต่อเนื่อง  ทำให้ได้เส้นโค้งสวยงาม สมมาตรซ้ายขวา

สำหรับการใช้งาน โดยทั่วไปสามารถเปลี่ยนจากคำสั่ง `px.box()` เป็น `px.violin()` ก็จะสร้าง Violin plot ได้เลย ดังตัวอย่างต่อไปนี้

In [ ]:
px.violin(iris,
       y='sepal_length',
       x='petal_length_group',
       color='class')

Violin plot มีพารามิเตอร์อื่น ๆ ที่สามารถกำหนดได้ ดังผลลัพธ์ในกราฟต่อไปนี้

In [ ]:
px.violin(iris,
       y='sepal_length',
       x='petal_length_group',
       color='class',
       points='all', # แสดงจุดข้อมูล
       box=True)     # แสดง box plot ใน violin plot

In [ ]:
px.violin(iris,
       y='sepal_length',
       x='petal_length_group',
       color='class',
       violinmode='overlay')     # violine plot ซ้อนกัน

**เพิ่มเติมเกี่ยวกับ Violin plot**
- ดูเว็บ Plotly https://plotly.com/python/violin/

## Histogram (กราฟการแจกแจงความถี่)

นักศึกษาได้ลองวาดกราฟโดยใช้ Histogram แล้วในเนื้อหาที่ผ่านมาในการวาดกราฟแท่ง  ดังนั้นในหัวข้อนี้ จะเพิ่มเติมเนื้อหาอีกบางส่วน

In [ ]:
import plotly.express as px
px.histogram(iris,
             x='petal_length',
             nbins=50)  # ลองปรับจำนวน bin

In [ ]:
px.histogram(iris,
             x='petal_length',
             nbins=50,
             marginal='violin', # **** ลองเปลี่ยนเป็น 'violin' และ 'rug'
             text_auto=True)

In [ ]:
px.histogram(iris,
             x='petal_length',
             nbins=50,
             marginal='box', # Box plot
             color='class')  # Histogram ย่อยตามสายพันธุ์

**หมายเหตุ** จากกราฟข้างต้น Box plot ที่ได้จะเห็นว่าบริเวณสองฝั่งของค่ามัธยฐาน จะมีการหักมุมเป็นร่องไว้ด้วย ส่วนที่เป็นของของร่องนี้ เรียกว่า **"notch"** ซึ่งแสดงถึงช่วงระดับความเชื่อมั่น 95% ที่ค่ามัธยฐานของกลุ่มประชากรจะตกอยู่ในช่วง notch นี้ (95% confidence interval of median)

In [ ]:
fig = px.histogram(iris,
             x='petal_length',
             nbins=50,
             marginal='box',
             color='class')

# ปรับสีโปรงแสง
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)

**ชื่อเรียกรูปร่างของ Histogram แบบต่าง ๆ (Mode of histogram)**

- สมมาตร (symmetric) และ unimodal
<img src="https://upload.wikimedia.org/wikipedia/commons/1/15/Symmetric-histogram.png">

- สมมาตร (symmetric)
<img src="https://upload.wikimedia.org/wikipedia/commons/7/7b/Symmetric2.png">

- เบ้ขวา (skewed right)
<img src="https://upload.wikimedia.org/wikipedia/commons/1/12/Skewed-right.png">

- เบ้ซ้าย (skewed left)
<img src="https://upload.wikimedia.org/wikipedia/commons/7/7b/Skewed-left.png">

- bimodal
<img src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bimodal-histogram.png">

- multimodal
<img src="https://upload.wikimedia.org/wikipedia/commons/3/39/Multimodal.png">

*ที่มา* https://en.wikipedia.org/wiki/Histogram

**ข้อเสียของ Histogram**
- ต้องกำหนดจำนวน bins เองให้เหมาะสม
- Histogram ใช้วาดการแจกแจงข้อมูลได้ทีละ Attribute ซึ่งในกรณีที่ข้อมูลมีหลาย Attribute ก็จะต้องใช้ Histogram หลายกราฟ ซึ่งทำความเข้าใจได้ยากขึ้น (ในประเด็นนี้ Box plot จะเข้าใจง่ายกว่า)

**ลิงค์ที่เกี่ยวข้องกับ Histogram**
- ตัวอย่างในเว็บ Plotly https://plotly.com/python/histograms/

## Scatter plot และ Scatter matrix

ทั้ง Box plot และ Histogram สามารถแสดงการแจกแจงของแต่ละ Attribute ได้เป็นอย่างดี แต่สิ่งที่มองข้ามไปคือความสัมพันธ์ระหว่าง Attribute ที่สามารถแสดงได้ด้วย Scatter plot ดังตัวอย่างต่อไปนี้

In [ ]:
px.scatter(iris,
       y='sepal_length',
       x='petal_length',
       color='class')

จากกราฟ มองแนวโน้มคร่าว ๆ ได้ดังนี้

- สำหรับ `Iris-versicolor` กับ `Iris-virginica` นั้น เมื่อ `petal_length` มีค่ามากก็มักจะมี `sepal_length` มากเช่นกัน และในทางกลับกัน หาก `petal_length` มีค่าน้อยก็มักจะมี `sepal_length` น้อย
- สำหรับ `Iris-setosa` นั้น `petal_length` ดูจะไม่เกี่ยวข้องกับ `sepal_length`

คำสั่ง `px.scatter()` แสดง Scatter plot ได้ทีละคู่ Attribute เท่านั้น หากต้องการแสดงหลายคู่ Attribute พร้อมกัน สามารถใช้คำสั่ง `px.scatter_matrix()` ได้ดังนี้

In [ ]:
px.scatter_matrix(
    iris,
    dimensions=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
    color='class')

## Correlation และ Correlation matrix

คำว่า **Correlation** หรือ**สหสัมพันธ์** เป็นคำศัพท์ทางสถิติ โดยในวิชานี้ เราหมายถึงค่าสหสัมพันธ์ของกลุ่มตัวอย่าง (Sample correlation) กล่าวคือไม่ใช่ค่าของประชากร (Population)

นิยามของสหสัมพันธ์มีหลายนิยาม เช่นนิยามของ Pearson, นิยามของ Spearman เป็นต้น แต่มีจุดประสงค์เดียวกันคือ ต้องการใช้เป็นค่าทางสถิติที่อธิบายความสัมพันธ์ระหว่าง Attribute 2 ค่า เช่น `sepal_width` vs. `sepal_length` โดยมีเงื่อนไขดังนี้

1. ถ้า Attribute หนึ่งมีค่ามาก แล้วอีก Attribute หนึ่งมีค่ามากด้วย หรือ ถ้า Attribute หนึ่งมีค่าน้อย แล้วอีก Attribute หนึ่งมีค่าน้อยด้วย ความสัมพันธ์แบบไปทางเดียวกันเช่นนี้ ก็ควรจะมีค่าสหสัมพันธ์ > 0 (เป็นบวก) หรือเรียกว่า **Positive correlation**
2. ถ้า Attribute หนึ่งมีค่ามาก แล้วอีก Attribute หนึ่งกลับมีค่าน้อย หรือ ถ้า Attribute หนึ่งมีค่าน้อย แล้วอีก Attribute หนึ่งกลับมีค่ามาก ความสัมพันธ์แบบสวนทางกันเช่นนี้ ก็ควรจะมีค่าสหสัมพันธ์ < 0 (เป็นลบ) หรือเรียกว่า **Negative correlation**
3. ถ้าในกรณีไม่มีความสัมพันธ์กัน ค่าสหสัมพันธ์ก็ควร = 0 หรือเรียกว่า **No correlation**
4. ค่าสหสัมพันธ์ต้องมีค่าอยู่ในช่วง -1 ถึง 1 ซึ่งการกำหนดเช่นนี้ก็เพื่อไม่ให้ระดับขนาดของค่า Attribute มามีผลต่อค่าสหสัมพันธ์ที่คำนวณได้ (กล่าวคือ สนใจเฉพาะทิศทางของความสัมพันธ์)

สำหรับในวิชานี้ จะศึกษาเฉพาะสหสัมพันธ์ตามนิยามของ Pearson



### Pearson's Correlation Coefficient

หากจะใช้ Pandas ในการคำนวณค่าสหสัมพันธ์แบบ Pearson จะใช้คำสั่งดังนี้

```python
DataFrame.corr(method='pearson', numeric_only=True)
```

ยกตัวอย่างเช่น หากพิจารณาข้อมูลเฉพาะดอกสายพันธ์ Setosa แล้ว จะเขียนโค้ดคำนวณได้ดังนี้

In [ ]:
setosa = iris[iris['class'] == 'Iris-setosa']
correlation = setosa.corr(method='pearson', numeric_only=True)
correlation

,sepal_length,sepal_width,petal_length,petal_width
sepal_length,1.000000,0.746780,0.263874,0.279092
sepal_width,0.746780,1.000000,0.176695,0.279973
petal_length,0.263874,0.176695,1.000000,0.306308
petal_width,0.279092,0.279973,0.306308,1.000000


ผลลัพธ์ข้างต้นเรียกว่า **Correlation matrix** ซึ่งเป็นตารางหรือ matrix ที่แสดงค่าสหสัมพันธ์ระหว่าง Attribute คู่หนึ่ง ๆ ยกตัวอย่างเช่น จากตาราง Correlation ระหว่าง sepal_width กับ sepal_length มีค่าเท่ากับ 0.746780 เป็นต้น

โดยแต่ละช่องในตาราง Correlation ข้างต้น เป็นตัวแทนของกราฟ 1 ช่องใน Scatter matrix ต่อไปนี้



In [ ]:
px.scatter_matrix(
    setosa, # เฉพาะข้อมูลสายพันธุ์ Setosa เท่านั้น
    dimensions=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
    color='class')

จากกราฟ สังเกตุได้ว่า

- ค่าสหสัมพันธ์ของคอลัมน์เดียวกัน จะมีค่าเท่ากับ 1 เสมอ
- `sepal_width` กับ `sepal_length` มีค่าสหสัมพันธ์เท่ากับ 0.746780 แต่ค่าของคู่ `sepal_length` กับ `petal_length` กลับมีค่า 0.263874 ซึ่งน้อยกว่า โดยสาเหตุที่น้อยกว่าก็สมเหตุสมผล เพราะจากกราฟคู่ `sepal_width` กับ `sepal_length` หากดูคร่าว ๆ จะเห็นแนวโน้มที่มีความชันของกราฟมากกว่า ค่าสหสัมพันธ์จึงมากกว่า

โดยปกติแล้ว หากค่า Correlation มีขนาด ดังนี้ (ไม่สนใจเครื่องหมายบวกลบ)
- 0.1 ถึง 0.3 เรากล่าวว่า มีความสัมพันธ์แบบ **Weak correlation**
- 0.3 ถึง 0.5 เรากล่าวว่า มีความสัมพันธ์แบบ **Moderate correlation**
- 0.5 ถึง 1.0 เรากล่าวว่า มีความสัมพันธ์แบบ **Strong correlation**


นอกจากนี้ เราสามารถแสดง Correlation matrix (เก็บในตัวแปรชื่อ `correlation` ข้างต้น) ด้วยกราฟประเภท **Heat map** ด้วยคำสั่ง `px.imshow()` (Image show) ซึ่งสามารถเรียกใช้ได้ ดังนี้

In [ ]:
px.imshow(correlation,
          text_auto='.2f')   # ก่อนหน้านี้เคยใช้ text_auto=True แต่ตอนนี้ใช้ '.2f' เพื่อแสดงทศนิยม 2 ตำแหน่ง

จะเห็นว่า กราฟ Heat map แสดงให้เห็นถึงความแตกต่างระหว่างค่าได้ดีกว่า


### นิยามทางคณิตศาสตร์ของ Pearson's Correlation Coefficient

แล้วค่า Correlation แบบ Pearson นี้ คำนวณมาอย่างไร

กำหนด Attribute $\mathbf{x}$ และ $\mathbf{y}$ ซึ่งคือคอลัมน์ $\mathbf{x}$ และ $\mathbf{y}$ ใน DataFrame นั่นเอง โดยที่แต่ละคอลัมน์มีค่าดังนี้

| $\mathbf{x}$ | $\mathbf{y}$ |
|:-- |:-- |
| $x_1$ | $y_1$ |
| $x_2$ | $y_2$ |
| $x_3$ | $y_3$ |
| $\vdots$ | $\vdots$ |
| $x_N$ | $y_N$ |

Pearson's Correlation Coefficient $\rho_{XY}$ ระหว่าง $\mathbf{x}$ และ $\mathbf{y}$คำนวณได้ดังนี้

$$
\begin{align*}
\mu_{\mathbf{x}} &= \frac{x_1+x_2+ \cdots + x_N}{N} \quad \text{คือค่าเฉลี่ยของคอลัมน์ $\mathbf{x}$} \\
\mu_{\mathbf{y}} &= \frac{y_1+y_2+ \cdots + y_N}{N} \quad \text{คือค่าเฉลี่ยของคอลัมน์ $\mathbf{y}$} \\
\sigma_{\mathbf{x}} &= \frac{1}{N-1}\sum_{i=1}^N (x_i -\mu_{\mathbf{x}})^2 \quad \text{คือ SD ของคอลัมน์ $\mathbf{x}$} \\
\sigma_{\mathbf{y}} &= \frac{1}{N-1}\sum_{i=1}^N (y_i -\mu_{\mathbf{y}})^2 \quad \text{คือ SD ของคอลัมน์ $\mathbf{y}$} \\
\rho_{XY} &= \frac{1}{N-1}\sum_{i=1}^N  \frac{ (x_i - \mu_{\mathbf{x}}) }{\sigma_{\mathbf{x}}} \frac{(y_i - \mu_{\mathbf{y}})}{\sigma_{\mathbf{y}}}
\end{align*}
$$

ซึ่งโค้ดต่อไปนี้ แสดงการคำนวณค่า Peason's correlation coefficient โดยตรงจากสูตร จะเห็นว่าค่าของคู่ `sepal_width` กับ `sepal_length` ตรงกับตาราง

In [ ]:
N = len(setosa)
x = setosa['sepal_width']
y = setosa['sepal_length']
mu_x = x.mean()   # สูตรหาค่า mean ของ Series
mu_y = y.mean()
sd_x = x.std()    # สูตรหาค่า SD ของ Series จะหารด้วย N-1
sd_y = y.std()
normalized_x = (x - mu_x)/sd_x   # การ Normalize หรือคือการลบค่าเฉลี่ยแล้วหารด้วย SD เป็นสูตรเดียวกันกับการหาค่า z-score
normalized_y = (y - mu_y)/sd_y
corr_xy = (normalized_x * normalized_y).sum()/(N-1)
print(f'Peason Correlation = {corr_xy}')

Peason Correlation = 0.7467803732639268


### ทำความเข้าใจค่าสหสัมพันธ์

กราฟต่อไปนี้แสดงถึงค่าสหสัมพันธ์ที่คำนวณได้จากข้อมูลต่าง ๆ

<img src="https://upload.wikimedia.org/wikipedia/commons/0/02/Correlation_examples.png">

จากกราฟ สิ่งที่น่าสนใจคือ
- $-1 \leq \rho_{\mathbf{X}\mathbf{Y}} \leq 1$
- สาเหตุที่รูปตรงกลางที่ไม่มีค่าสหสัมพันธ์แสดงไว้ เนื่องจากว่าไม่สามารถคำนวณค่าสหสัมพันธ์ได้ เพราะว่าค่าส่วนเบี่ยงเบนมาตรฐานของแกนตั้งเป็นศูนย์
- ในแถวที่สอง เป็นข้อมูลที่อยู่บนเส้นตรงพอดิบพอดี ดังนั้นจึงมีค่าสหสัมพันธ์ = 1 หรือ -1 หรือหาค่าไม่ได้
- **สำคัญ:** ในแถวสุดท้ายมีค่าสหสัมพันธ์เท่ากับ 0 ทั้งหมด แสดงให้เห็นว่า Pearson's correlation coefficient ใช้วัดได้แต่ความสัมพันธ์เชิงเส้นเท่านั้น




### ค่าสถิติเดียวกันแต่คนละกราฟ

พิจารณาข้อมูลต่อไปนี้ เป็นข้อมูลที่มีค่าทางสถิติเหมือนกันทุกกราฟ ทั้งค่าเฉลี่ย ค่าส่วนเบี่ยงเบนมาตรฐาน และค่าสหสัมพันธ์ แต่กลับมีภาพลักษณ์ต่างกันสิ้นเชิง ดังนั้นจึงเป็นสิ่งที่ควรระลึกถึงเสมอถึงข้อจำกัดของค่าสถิติเชิงพรรณนา (Descriptive statistic)

ที่มา: https://www.research.autodesk.com/publications/same-stats-different-graphs/

#### Anscombe’s Quartet

ภาพครึ่งซ้าย (Anscombe's Quartet) แม้ว่าจะดูออกว่าเป็นคนละกราฟ แต่กลับมีค่าสถิติเท่ากัน ส่วนภาพครึ่งขวา (Unstructured Quartet) ก็มีค่าสถิติเท่ากันเช่นกัน ตัวกราฟดูไม่แตกต่างเท่าไร เพราะจุดข้อมูลกระจัดกระจาย

<img src="https://www.research.autodesk.com/app/uploads/2023/03/Anscombe-2.png">


#### The Datasaurus Dozon

กราฟไดโนเสาร์ ก็มีค่าทางสถิติเหมือนกับกราฟรูปอื่น ๆ [คลิกลิงค์ เพื่อดูภาพเคลื่อนไหว](https://www.research.autodesk.com/app/uploads/2023/03/DinoSequential-1.gif)

<img src="https://www.research.autodesk.com/app/uploads/2023/03/AllDinos-2.png">



### Correlation does not imply causation

ในทางสถิติ ค่าสหสัมพันธ์ที่เรากล่าวถึงวันนี้ มีประโยคยอดฮิตที่มักจะมาคู่กันเสมอ คือประโยคที่ว่า "Correlation does not imply causation."

นักศึกษาคิดว่าประโยคดังกล่าวมีความหมายว่าอะไร จงยกตัวอย่างในชีวิตจริง (ลอง Google ดูก็ได้)